# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [ ]:
from IPython.display import display, Markdown, update_display
import os
from dotenv import load_dotenv
from openai import OpenAI
import requests

In [ ]:
# Cargar las variables del fichero .env
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Mostrar resultados con estilo
if not api_key:
    display(Markdown("### ⚠️ **Clave API no encontrada**\n"
                     "Por favor, verifica el archivo .env en la carpeta de este proyecto y asegúrate de que contenga tu clave API de OpenAI. "
                     "Consulta el cuaderno de resolución de problemas si necesitas ayuda."))
elif not api_key.startswith("sk-proj-"):
    display(Markdown("### 🔑 **Clave API detectada, pero parece incorrecta**\n"
                     "- La clave no comienza con sk-proj-. Por favor, verifica si copiaste la clave completa desde OpenAI.\n"
                     "- Consulta el cuaderno de resolución de problemas para más información."))
elif api_key.strip() != api_key:
    display(Markdown("### ✂️ **Clave API detectada con posibles espacios**\n"
                     "- Parece que la clave tiene espacios adicionales o tabulaciones al inicio o al final. "
                     "Por favor, edita el archivo .env y asegúrate de que la clave esté limpia."))
else:
    display(Markdown("### ✅ **¡Clave API encontrada y válida!**\n"
                     "¡Todo listo! Puedes proceder con las llamadas a la API de OpenAI. 🚀"))

In [ ]:
# constantes

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
LLAMA_URL = "http://localhost:11434"

In [ ]:
# Aquí está la pregunta; escribe sobre esto para preguntar algo nuevo

question = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Configuramos OpenAI
openai = OpenAI()

In [ ]:
def get_openai_response_stream(model, question):
    """
    Obtiene la respuesta de OpenAI con streaming, procesando el contenido en tiempo real.
    """
    stream = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Eres un asistente experto en programación."},
            {"role": "user", "content": question}
        ],
        stream=True  # Activar streaming
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        text_chunk = chunk.choices[0].delta.content or ""
        response += text_chunk
        # Eliminar triples backticks y palabras clave innecesarias
        cleaned_response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(cleaned_response), display_id=display_handle.display_id)

    return response.strip()

In [ ]:
def get_llama_response(model, question, url):
    """
    Obtiene la respuesta de Llama y la procesa.
    """
    llama_response = requests.post(
        f"{url}/v1/completions",
        json={
            "model": model,
            "prompt": question
        }
    )
    if llama_response.status_code == 200:
        llama_result = llama_response.json()
        return llama_result.get('choices', [{}])[0].get('text', '').strip()
    else:
        return "Error: Llama no respondió correctamente."

In [ ]:
def process_responses(model_openai, model_llama, question, llama_url):
    """
    Llama a las APIs de OpenAI y Llama de forma secuencial.
    """

    # Mostrar progreso inicial
    display(Markdown("### 🧠 **Iniciando el proceso...**"))

    # 1. OpenAI: Mostrar título y realizar la solicitud
    display(Markdown(f"#### 🧠 **Llamando a OpenAI ({model_openai})...**"))
    openai_response = get_openai_response_stream(model_openai, question)

    # 2. Llama: Mostrar título y realizar la solicitud
    display(Markdown(f"#### 🐎 **Llamando a Llama ({model_llama})...**"))
    llama_response = get_llama_response(model_llama, question, llama_url)

    # Mostrar ambas respuestas
    display(Markdown("### 🆚 **Comparación de respuestas**"))
    display(Markdown(f"#### 🧠 **OpenAI ({model_openai}):**\n\n{openai_response}\n"))
    display(Markdown(f"#### 🐎 **Llama ({model_llama}):**\n\n{llama_response}\n"))

    return openai_response, llama_response

In [ ]:
# Llamar al método principal para procesar ambas respuestas
openai_result, llama_result = process_responses(MODEL_GPT, MODEL_LLAMA, question, LLAMA_URL)